In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [ ]:
# Load Data (replace this with actual CSV file or DataFrame)
data = pd.read_csv('merged_properties.csv')  # Replace 'your_file.csv' with your actual file path
# Features and targets
# X = data.drop(columns=['c44', 'e15', 'q15', 'μ11', 'ϵ11', 'α11', 'ρ']).values
X = data[['vf', 'c55e', 'e15e', 'q15e', 'ϵ11e', 'μ11e', 'α11e', 'ρe', 'c55f', 'e15f', 'q15f', 'ϵ11f', 'μ11f', 'α11f', 'ρf']].values
y = data[['c44', 'e15', 'q15', 'μ11', 'ϵ11', 'α11', 'ρ']].values

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")


Shape of X: (1045, 15)
Shape of y: (1045, 7)


In [39]:
# Initialize empty lists to store the exponents for X and y
exponents_X = []
exponents_y = []

# Create copies of X and y to store the transformed data
formal_X = X.copy()
formal_y = y.copy()

# Process X
for i in range(X.shape[1]):
    # Calculate the mean of the column
    mean_value = X[:, i].mean()
    
    # Extract the exponent from the scientific notation of the mean value
    exponent = int(f"{mean_value:.1e}".split('e')[1])
    exponents_X.append(exponent)
    
    # Divide all values in the column by 10^exponent and round to 7 decimal places
    formal_X[:, i] = (X[:, i] / (10 ** exponent)).round(7)

# Process y
for i in range(y.shape[1]):
    # Calculate the mean of the column
    mean_value = y[:, i].mean()
    
    # Extract the exponent from the scientific notation of the mean value
    exponent = int(f"{mean_value:.1e}".split('e')[1])
    exponents_y.append(exponent)
    
    # Divide all values in the column by 10^exponent and round to 7 decimal places
    formal_y[:, i] = (y[:, i] / (10 ** exponent)).round(7)

# Print the updated arrays and the lists of exponents
print("Exponents for X:", exponents_X)
print("Exponents for y:", exponents_y)

Exponents for X: [-1, 10, 1, 0, -7, -6, 0, 3, 11, 0, 2, -2, -3, -10, 3]
Exponents for y: [10, 0, 1, -4, -2, -9, 3]


In [40]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(formal_X)
y_scaled = scaler_y.fit_transform(formal_y)

In [41]:

# Split first
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# scalers_X = []
# X_train_scaled = np.zeros_like(X_train)
# X_test_scaled = np.zeros_like(X_test)

# for i in range(X.shape[1]):
#     scaler = MinMaxScaler()
#     X_train_scaled[:, i] = scaler.fit_transform(X_train[:, i].reshape(-1, 1)).flatten()
#     X_test_scaled[:, i] = scaler.transform(X_test[:, i].reshape(-1, 1)).flatten()
#     scalers_X.append(scaler)

# # Per target scaling
# scalers_y = []
# y_train_scaled = np.zeros_like(y_train)
# y_test_scaled = np.zeros_like(y_test)

# for i in range(y_train.shape[1]):
#     scaler = MinMaxScaler()
#     y_train_scaled[:, i] = scaler.fit_transform(y_train[:, i].reshape(-1, 1)).flatten()
#     y_test_scaled[:, i] = scaler.transform(y_test[:, i].reshape(-1, 1)).flatten()
#     scalers_y.append(scaler)

In [43]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(15,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(7, activation='linear')  # 7 target outputs
])

# Compile with standard MSE
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Training
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=300,
    batch_size=4,
    verbose=1
)

Epoch 1/300


/opt/anaconda3/envs/Thesis/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 830us/step - loss: 1.0260 - mae: 0.3649 - val_loss: 0.6358 - val_mae: 0.1991
Epoch 2/300
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - loss: 0.5786 - mae: 0.1790 - val_loss: 0.5348 - val_mae: 0.1936
Epoch 3/300
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step - loss: 0.4993 - mae: 0.1791 - val_loss: 0.4667 - val_mae: 0.1968
Epoch 4/300
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step - loss: 0.2207 - mae: 0.1258 - val_loss: 0.4892 - val_mae: 0.1405
Epoch 5/300
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step - loss: 0.9882 - mae: 0.1752 - val_loss: 0.4101 - val_mae: 0.1330
Epoch 6/300
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step - loss: 0.4082 - mae: 0.1255 - val_loss: 0.4075 - val_mae: 0.1172
Epoch 7/300
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - loss: 0.4461 - mae: 0.1262 - val_loss: 0.4113 - val_mae: 0.1139
Epoch 8/300
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step - loss: 0.2974 - mae: 0.1047 - val_loss: 0.4312 - val_mae: 0.1287
Epoch 9/300
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [44]:
# Evaluate model
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MAE: {mae:.4f}")

Test MAE: 0.0837


In [45]:
y_pred = model.predict(X_test)

# Inverse scaling for y_pred and y_test
y_pred_original = scaler_y.inverse_transform(y_pred)
y_test_original = scaler_y.inverse_transform(y_test)

np.set_printoptions(linewidth=np.inf)  # Set the print options to avoid line breaks
for i in range(5):
    print(f"Predicted: {y_pred_original[i]}")
    print(f"Actual:    {y_test_original[i]}")
    print("-" * 50)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Predicted: [ 3.8434527  -1.1806159  11.235526    0.238786   -0.15530989 -0.17421177  7.329519  ]
Actual:    [ 5.75659230e+00  3.79153000e-02  1.01572435e+01  3.00662100e-01  0.00000000e+00 -6.17500000e-04  7.33000000e+00]
--------------------------------------------------
Predicted: [ 8.9802475e+00  1.9519508e+00 -7.7989376e-01  7.2907939e+00 -4.5597181e-03  1.3536985e+00  7.0633774e+00]
Actual:    [9.6557162e+00 5.6567740e-01 1.4630000e-04 7.1873362e+00 1.0000000e-07 2.8740900e-01 7.1000000e+00]
--------------------------------------------------
Predicted: [ 4.5754986   6.360233    0.04457579 32.081406    0.03919689 -0.12595204  5.295139  ]
Actual:    [3.06869110e+00 3.47900000e-04 9.99999998e-08 3.18686247e+01 0.00000000e+00 7.08191700e-01 5.28000000e+00]
--------------------------------------------------
Predicted: [ 26.207727   125.388176    23.664383     0.21520497  13.834323   -45.986965     8.082935  ]
Actual:    [ 12.3707914  69.7810465  17

In [ ]:
exponents_y

[10, 0, 1, -4, -2, -9, 3]

In [38]:
# -------- Prediction Function --------
def predict_custom_input(input_list):
    """
    input_list: list of 15 feature values in order
    """
    if len(input_list) != 15:
        raise ValueError("Input must be a list of 15 feature values.")
    
    input_scaled = scaler_X.transform([input_list])
    prediction_scaled = model.predict(input_scaled, verbose=0)
    prediction = scaler_y.inverse_transform(prediction_scaled)
    
    # Display the results
    target_columns = [
        "c55", "e15", "q15",
        "ϵ11", "μ11", "α11", "ρ"
    ]
    result = dict(zip(target_columns, prediction[0]))
    print("Predicted Properties:")
    for k, v in result.items():
        print(f"{k}: {v:.4f}")
    return prediction[0]

# -------- Example of using predict_custom_input --------
# Example Input (replace these with actual values when running)
example_input = [
    0.05, 44000000000.0, 11.4, 0.0, 9.82e-09, 5e-06, 0, 5700, 45300000000.0, 0, 550.0, 8.000000000000001e-11, 0.000157, 0.0, 5300
]

example_output = [
    47665708776.4155, 0.015891104909231, 496.3562235447875, 0.0001422813709311, 9.727373332703483e-11, -4.046667731750246e-12, 5320.0
]

# Get the prediction
predicted_output = predict_custom_input(example_input)

# Compare with example_output
print("\nComparison with Example Output:")
for i, (predicted, example) in enumerate(zip(predicted_output, example_output)):
    print(f"Target {i + 1}: Predicted = {predicted:.4f}, Example = {example:.4f}, Difference = {abs(predicted - example):.4f}")

Predicted Properties:
c55: 91987156992.0000
e15: -4.3636
q15: 430.8117
ϵ11: 0.0001
μ11: 0.0043
α11: 0.0000
ρ: 5269.8472

Comparison with Example Output:
Target 1: Predicted = 91987156992.0000, Example = 47665708776.4155, Difference = 44321447936.0000
Target 2: Predicted = -4.3636, Example = 0.0159, Difference = 4.3795
Target 3: Predicted = 430.8117, Example = 496.3562, Difference = 65.5445
Target 4: Predicted = 0.0001, Example = 0.0001, Difference = 0.0000
Target 5: Predicted = 0.0043, Example = 0.0000, Difference = 0.0043
Target 6: Predicted = 0.0000, Example = -0.0000, Difference = 0.0000
Target 7: Predicted = 5269.8472, Example = 5320.0000, Difference = 50.1528
